In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
%%writefile app.py
import streamlit as st
import time
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# FUNCTIONS & GLOBAL VARS
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
# generation_kwargs = {
#     "max_length": 512,
#     "min_length": 64,
#     "no_repeat_ngram_size": 3,
#     "early_stopping": True,
#     "num_beams": 5,
#     "length_penalty": 1.5,
# }
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generate_recipe(ingredients):
    texts = ""
    for key in ingredients:
        if ingredients[key]:
            texts = texts + key + ", "

    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe


# PAGE SETTINGS
st.set_page_config(page_title="Fridge Feast", page_icon=':pizza:', layout="wide")

# PAGE TITLE AND OPENING
with st.container():
    title_cols = st.columns([6,1])
    with title_cols[0]:
        st.title("FRIDGE FEAST")
        st.text("Welcome to Fridge Feast!\n"
                "This is a quick & easy tool that will help you FEAST with simple ingredients you have in your fridge or pantry.\n"
                "Fill out the ingredients you would like to use down bellow, click Submit and start cooking!")
        st.text("")
    with title_cols[1]:
        st.text("")
        st.image("https://img.hotimg.com/fridge_feast_good-transformed-1.png", use_column_width=True)

# FORM
with st.container():
    with st.form("ingredients_form"):
        # CREATING INGREDIENTS DICT AND SPLITTING THE FORM TO INPUT AND OUTPUT
        ingredients = {}
        header_cols = st.columns([1,1,1.5], gap="small")

        # LEFT INPUT
        with header_cols[0]:
            # PROTEIN
            pro_cont = st.container(border=True)
            pro_cont.subheader("Protein 💪")
            pro_cols = pro_cont.columns(2)
            ingredients["egg"] = pro_cols[0].checkbox("Egg🥚")
            ingredients["tofu"] = pro_cols[0].checkbox("Tofu🌱")
            ingredients["shrimp"] = pro_cols[0].checkbox("Shrimp🍤")
            ingredients["chicken"] = pro_cols[1].checkbox("Chicken🍗")
            ingredients["beef"] = pro_cols[1].checkbox("Beef🥩")
            ingredients["salmon"] = pro_cols[1].checkbox("Fish🐟")

            # DAIRY
            dairy_cont = st.container(border=True)
            dairy_cont.subheader("Dairy 🐮")
            dairy_cols = dairy_cont.columns(2)
            ingredients["butter"] = dairy_cols[0].checkbox("Butter🧈")
            ingredients["cream"] = dairy_cols[0].checkbox("Cream🍶")
            ingredients["cheese"] = dairy_cols[1].checkbox("Cheese🧀")
            ingredients["yogurt"] = dairy_cols[1].checkbox("Yogurt🥛")


        # RIGHT INPUT
        with header_cols[1]:
            # CARBS
            carb_cont = st.container(border=True)
            carb_cont.subheader("Carbs 🥨")
            carb_cols = carb_cont.columns(2)
            ingredients["bread"] = carb_cols[0].checkbox("Bread🍞")
            ingredients["potato"] = carb_cols[0].checkbox("Potato🥔")
            ingredients["pasta"] = carb_cols[1].checkbox("Pasta🍝")
            ingredients["rice"] = carb_cols[1].checkbox("Rice🍚")

            # VEGGIES
            veg_cont = st.container(border=True)
            veg_cont.subheader("Veggies 🥬")
            veg_cols = veg_cont.columns([1, 1.2])
            ingredients["onion"] = veg_cols[0].checkbox("Onion🧅")
            ingredients["garlic"] = veg_cols[1].checkbox("Garlic🧄")
            ingredients["tomato"] = veg_cols[0].checkbox("Tomato🍅")
            ingredients["mushroom"] = veg_cols[1].checkbox("Mushroom🍄")
            ingredients["bell pepper"] = veg_cols[1].checkbox("Bell Pepper🫑")
            ingredients["carrot"] = veg_cols[0].checkbox("Carrot🥕")



        # Every form must have a submit button.
        submitted = header_cols[0].form_submit_button("Submit", use_container_width=True)
        header_cols[1].subheader(":red[➜➜] **:red[FEAST]** 🍽️🥂🍾🎊")

        if submitted and any(ingredients.values()):
            # OUTPUT
            with header_cols[2]:
                with st.spinner('Cooking up a feast... be right back!'):
                    generated = generate_recipe(ingredients)
                    for text in generated:
                        sections = text.split("\n")
                        for section in sections:
                            section = section.strip()
                            if section.startswith("title:"):
                                section = section.replace("title:", "")
                                headline = "TITLE"
                            elif section.startswith("ingredients:"):
                                section = section.replace("ingredients:", "")
                                headline = "INGREDIENTS"
                            elif section.startswith("directions:"):
                                section = section.replace("directions:", "")
                                headline = "DIRECTIONS"

                            if headline == "TITLE":
                                st.markdown(f"**{headline}:** {section.strip().capitalize()}")
                            elif headline == "INGREDIENTS":
                                section_info = [f"  - {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                                st.markdown(f"**{headline}:**")
                                st.write("\n".join(section_info))
                            else:
                                section_info = [f"  {i+1}. {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                                st.markdown(f"**{headline}:**")
                                st.write("\n".join(section_info))

                        print("-" * 130)
        else:
            # WAITING FOR USER SUBMISSION
            with header_cols[2]:
                st.header("🧑🏻‍🍳Waiting for Ingredients👩🏾‍🍳")


Writing app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

# copy this code

35.243.197.92


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

# click on the 3rd link and paste the code you copied to access the website




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.243.197.92:8501

npx: installed 22 in 5.264s
your url is: https://soft-falcons-lie.loca.lt
tokenizer_config.json: 100% 1.92k/1.92k [00:00<00:00, 6.28MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 10.4MB/s]
special_tokens_map.json: 100% 1.79k/1.79k [00:00<00:00, 7.43MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100% 1.56k/1.56k [00:00<00:00, 6.84MB/s]
flax_model.msgpack: 100% 892M/892M [00:14<00:00, 59.8MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
----------------------------------------------------------------------------------------------------------------------------------
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


EXTRA CODE
IRRELEVENT TO THE WEBSITE

In [ ]:
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
# generation_kwargs = {
#     "max_length": 512,
#     "min_length": 64,
#     "no_repeat_ngram_size": 3,
#     "early_stopping": True,
#     "num_beams": 5,
#     "length_penalty": 1.5,
# }
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}


special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
items = [
    "butter, beef, rice, onion, garlic, nushroom"
]
generated = generation_function(items)
for text in generated:
    sections = text.split("\n")
    for section in sections:
        section = section.strip()
        if section.startswith("title:"):
            section = section.replace("title:", "")
            headline = "TITLE"
        elif section.startswith("ingredients:"):
            section = section.replace("ingredients:", "")
            headline = "INGREDIENTS"
        elif section.startswith("directions:"):
            section = section.replace("directions:", "")
            headline = "DIRECTIONS"

        if headline == "TITLE":
            print(f"[{headline}]: {section.strip().capitalize()}")
        else:
            section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
            print(f"[{headline}]:")
            print("\n".join(section_info))

    print("-" * 130)


[TITLE]: Beef risotto
[INGREDIENTS]:
  - 1: 4 tablespoons butter
  - 2: 1 pound beef
  - 3: 1 cup rice
  - 4: 1 whole onion
  - 5: 3 cloves garlic
  - 6: 1/2 cups nushroom, sliced
[DIRECTIONS]:
  - 1: Melt 1 tablespoon butter in pan and brown beef.
  - 2: Remove beef.
  - 3: Saute diced onion and minced garlic in the remaining butter until onions become clear.
  - 4: Then add the rice and saute for about a minute.
  - 5: Put the beef back in and add the nushrooms.
  - 6: Simmer for about 5 minutes.
  - 7: Then put the sauteed vegetables on top.
  - 8: Let the risotto thicken slightly.
  - 9: Serve hot.
----------------------------------------------------------------------------------------------------------------------------------
